# BDM HW2_dcr346
Student: Dana Chermesh Reshef, July 2018
### task:
Compute the number of restaurants per each cuisine using the provided **nyc_restaurant_inspections.csv** data set. The CSV file is available for download under the Data Sets page on NYU Classes. The cuisine type can be extracted from the _"CUISINE DESCRIPTION"_ column. Note that, a single restaurant may be inspected multiple times. You must use the _"CAMIS"_ column to keep only unique restaurants while computing the number of restaurants per cuisine.

You MUST turn in a Spark notebook, AND a Spark script that works on the NYU Dumbo cluster. For your script, the input and out file path should be specified through the command line. Basically, I should be able to run your script as:

`spark-submit YOUR_SCRIPT.py nyc_restaurant_inspections.csv output_folder`

and it will produce an output on HDFS under output_folder.
Using "hadoop fs -getmerge" or through the notebook, I should get the expected output similar to the below:

```
[('American', 6002),
 ('Chinese', 2399),
 ('Cafe', 1629),
 ('Other', 1296),
 ('Pizza', 1186),
 ('Italian', 1016),
 ('Mexican', 877),
 ('Japanese', 859),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 840),
 ('Bakery', 733),
 ('Caribbean', 671),
 ('Spanish', 644),
 ('Donuts', 537),
 ('Pizza/Italian', 483),
 ('Chicken', 456),
 ('Sandwiches', 406),
 ('Juice, Smoothies, Fruit Salads', 382),
 ('Hamburgers', 378),
 ('Asian', 371),
 ('Ice Cream, Gelato, Yogurt, Ices', 339),
 ('Indian', 332),
 ('Jewish/Kosher', 327),
 ('French', 319),
 ('Delicatessen', 294),
 ('Thai', 286)]
```

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
NYC_CUIS = 'nyc_restaurant_inspections.csv'

In [3]:
# reading NYC_CUIS to a RDD. 
# use_unicode can be changed accordingly to your data file to handle Unicode
# If you cannot parse your data due to an 'utf8' or 'ascii' decoding issue, it might
# be a good thing to try flipping the use_unicode parameter here.

cuis = sc.textFile(NYC_CUIS, use_unicode=True).cache()

# list the column index and column names to see
# which column we need to use for our task. 
# In this case, I'm interested in the 'CAMIS' (#0) and the 'CUISINE DESCRIPTION' (#7).
list(enumerate(cuis.first().split(',')))

[(0, 'CAMIS'),
 (1, 'DBA'),
 (2, 'BORO'),
 (3, 'BUILDING'),
 (4, 'STREET'),
 (5, 'ZIPCODE'),
 (6, 'PHONE'),
 (7, 'CUISINE DESCRIPTION'),
 (8, 'INSPECTION DATE'),
 (9, 'ACTION'),
 (10, 'VIOLATION CODE'),
 (11, 'VIOLATION DESCRIPTION'),
 (12, 'CRITICAL FLAG'),
 (13, 'SCORE'),
 (14, 'GRADE'),
 (15, 'GRADE DATE'),
 (16, 'RECORD DATE'),
 (17, 'INSPECTION TYPE')]

In [4]:
# using mapPartitionsWithIndex() to check if we're hitting the header
# (aka the first partition). If so, we just skip the first row.

def extractRest(partId, records):
    if partId==0:
        next(records)
    import csv
    reader = csv.reader(records)
    for row in reader:
        (camis, cuisine) = (row[0], row[7])
        yield (camis, cuisine)

nycResturants = cuis.mapPartitionsWithIndex(extractRest)
nycResturants.take(5)

[('50015075', 'Chicken'),
 ('40364529', 'Jewish/Kosher'),
 ('41580756', 'Caribbean'),
 ('50001255', 'Italian'),
 ('50016437', 'Korean')]

In [9]:
nycCuisine = nycResturants \
            .map(lambda x: (x, 1)) \
            .reduceByKey(lambda x,y: 1) \
            .keys().map(lambda x: (x[1],1)) \
            .reduceByKey(lambda x,y: x+y) \
            .top(25, key=lambda x: x[1])

nycCuisine

[('American', 6002),
 ('Chinese', 2399),
 ('CafÃ©/Coffee/Tea', 1629),
 ('Other', 1296),
 ('Pizza', 1186),
 ('Italian', 1016),
 ('Mexican', 877),
 ('Japanese', 859),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 840),
 ('Bakery', 733),
 ('Caribbean', 671),
 ('Spanish', 644),
 ('Donuts', 537),
 ('Pizza/Italian', 483),
 ('Chicken', 456),
 ('Sandwiches', 406),
 ('Juice, Smoothies, Fruit Salads', 382),
 ('Hamburgers', 378),
 ('Asian', 371),
 ('Ice Cream, Gelato, Yogurt, Ices', 339),
 ('Indian', 332),
 ('Jewish/Kosher', 327),
 ('French', 319),
 ('Delicatessen', 294),
 ('Thai', 286)]